<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/File_Utilities_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# %%writefile FileUtilities.py
import glob, os, pathlib, sys
from os.path import *
from pathlib import Path

class ColorOutput:
    LITE = '\033[96m'; HEADER = '\033[95m'; BLUE = '\033[94m';
    WARNING = '\033[93m'; GREEN = '\033[92m'; RED = '\033[91m'; 
    DARK = '\033[90m'; UNDERLINE = '\033[4m';BOLD = '\033[1m'; 
    WHITE = '\033[0m'
global C
C = ColorOutput()

class AllFileUtilities(ColorOutput):
    def __init__(self):
        self.C = ColorOutput
        self.homePath = Path.home()
        self.cwdPath = Path.cwd()
        self.allFiles = glob.glob('**', recursive=True)
        self.allFilesDict = {}

    def listFiles(self):
        tempDict = self.allFilesDict
        fileCount = 0
        print(f'{C.BLUE}{self.cwdPath}')
        for fil in self.allFiles:
            fullPath = Path(realpath(fil)) # POSIX path
            realPath = realpath(fullPath) # String path
            if isdir(fullPath) and not realPath.__contains__('__pycache__'):  
                setSystemPaths(newSysPath=realPath)
                print(f'\n{C.BLUE}{fullPath}')
            elif isfile(fullPath) and not realPath.__contains__('__pycache__'): 
                fileCount += 1
                tempDict[fileCount] = fullPath
                print(f'{C.WHITE}{fileCount}. {fullPath.name}') 

    def inspectFile(self, selection):
        if selection <= len(self.allFilesDict):  
            try:
                pth = self.allFilesDict[selection]  
                with open(pth, 'r') as openFile:  
                    print(f'# %%writefile {pth.name}\n')  
                    print(openFile.read())

            except:
                print(f'{C.RED}BULLSHIT')

    def setSystemPaths(self, newSysPath=None):
        if newSysPath == None:
            print(f'{C.RED}newSysPath == {newSysPath}') 
        elif newSysPath :
            pathList = sys.path.copy()
            if not newSysPath in pathList:
                print(f'{C.GREEN}Ok Go Fix')

        pass

    def printSystemPaths(self):
        pathList = sys.path.copy() 
        pathCount = 0
        for pth in pathList:
            pathCount += 1
            print(f'{C.WHITE}{pathCount}. {C.BLUE}{pth}')

afu = AllFileUtilities()
afu.setSystemPaths(newSysPath=None)

newSysPath == None


In [ ]:
afu.inspectFile(selection= 1 )

In [ ]:
for item in afu.allFilesDict:
    fullPath = afu.allFilesDict[item]
    print(f'{C.BLUE}{fullPath.name}\n{fullPath}\n')